<a href="https://colab.research.google.com/github/svechino/TravelTide/blob/main/All_Perks_%26_Users_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from datetime import date

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# Part 1. Processing data on calculated indexes

In [ ]:
# Loading the data from csv file that we've got in the previous step SQL analysis:
indexes = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/all_perks.csv')
indexes.head()

,user_id,hotel_hunter_index,average_bags_scaled,cancellation_rate_scaled,bargain_hunter_index,combined_booking_scaled,session_intencity_index
0,23557,0.00268,NaN,0.0,NaN,0.0,0.333525
1,94883,NaN,0.10,0.0,NaN,1.0,0.418203
2,101486,NaN,0.00,0.0,NaN,0.5,0.549539
3,101961,NaN,0.08,0.0,0.000058,1.0,0.176267
4,106907,NaN,1.00,0.5,NaN,1.0,0.434310


In [ ]:
# Checking formats of our data:
indexes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5998 entries, 0 to 5997
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   user_id                   5998 non-null   int64  
 1   hotel_hunter_index        1646 non-null   float64
 2   average_bags_scaled       5178 non-null   float64
 3   cancellation_rate_scaled  5542 non-null   float64
 4   bargain_hunter_index      1718 non-null   float64
 5   combined_booking_scaled   5542 non-null   float64
 6   session_intencity_index   5998 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 328.1 KB



### **Approach to Customer Segmentation:**


1.   We will begin by ranking all customers for each of the perks based on their respective indexes in descending order.


2.   After ranking, we will determine the minimum rank for each customer, and label them according to the perk associated with the minimum rank.

3. We will create a function to determine the rank based on the minimum rank encountered

4. In cases where customers have multiple minimum ranks, we will assign them the label corresponding to the first minimum rank encountered.

5. Customers who have shown no activity in booking and possess all NaN values for indexes will be categorized as "Active Searchers." We recommend keeping these customers in this segment for a period to better understand their preferences before making perk suggestions. We will also include in this group customers whose minimum rank is Session Activity, so that we will have customers with non-null indexes. This will help us gain insights into the preferences of this segment.

In [ ]:
# Step 1. Ranking all our indexes in descending order, keeping NaNs:
indexes['rank_hotel_hunter'] = indexes['hotel_hunter_index'].rank(na_option = 'keep', ascending=False)
indexes['rank_ave_bags'] = indexes['average_bags_scaled'].rank(na_option = 'keep', ascending=False)
indexes['rank_cancellation_rank'] = indexes['cancellation_rate_scaled'].rank(na_option = 'keep', ascending=False)
indexes['rank_bargain_hunter'] = indexes['bargain_hunter_index'].rank(na_option = 'keep', ascending=False)
indexes['rank_combined_booking'] = indexes['combined_booking_scaled'].rank(na_option = 'keep', ascending=False)
indexes['rank_session_activity'] = indexes['session_intencity_index'].rank(na_option = 'keep', ascending=False)
indexes.head(5)

,user_id,hotel_hunter_index,average_bags_scaled,cancellation_rate_scaled,bargain_hunter_index,combined_booking_scaled,session_intencity_index,rank_hotel_hunter,rank_ave_bags,rank_cancellation_rank,rank_bargain_hunter,rank_combined_booking,rank_session_activity
0,23557,0.00268,NaN,0.0,NaN,0.0,0.333525,263.0,NaN,3069.0,NaN,5283.5,4866.5
1,94883,NaN,0.10,0.0,NaN,1.0,0.418203,NaN,2866.5,3069.0,NaN,1233.0,2687.0
2,101486,NaN,0.00,0.0,NaN,0.5,0.549539,NaN,4635.0,3069.0,NaN,4343.5,310.0
3,101961,NaN,0.08,0.0,0.000058,1.0,0.176267,NaN,3380.0,3069.0,1679.0,1233.0,5988.0
4,106907,NaN,1.00,0.5,NaN,1.0,0.434310,NaN,4.0,52.0,NaN,1233.0,2231.0


In [ ]:
# Step 2. Finding minimum rating for each user
indexes['min_c_h'] = indexes[['rank_hotel_hunter',
                              'rank_ave_bags',
                              'rank_cancellation_rank',
                              'rank_bargain_hunter',
                              'rank_combined_booking',
                              'rank_session_activity']].min(axis=1)
indexes.head()

,user_id,hotel_hunter_index,average_bags_scaled,cancellation_rate_scaled,bargain_hunter_index,combined_booking_scaled,session_intencity_index,rank_hotel_hunter,rank_ave_bags,rank_cancellation_rank,rank_bargain_hunter,rank_combined_booking,rank_session_activity,min_c_h
0,23557,0.00268,NaN,0.0,NaN,0.0,0.333525,263.0,NaN,3069.0,NaN,5283.5,4866.5,263.0
1,94883,NaN,0.10,0.0,NaN,1.0,0.418203,NaN,2866.5,3069.0,NaN,1233.0,2687.0,1233.0
2,101486,NaN,0.00,0.0,NaN,0.5,0.549539,NaN,4635.0,3069.0,NaN,4343.5,310.0,310.0
3,101961,NaN,0.08,0.0,0.000058,1.0,0.176267,NaN,3380.0,3069.0,1679.0,1233.0,5988.0,1233.0
4,106907,NaN,1.00,0.5,NaN,1.0,0.434310,NaN,4.0,52.0,NaN,1233.0,2231.0,4.0


In [ ]:
# Step 3. Creating function for defining the most attractive perk for each user:
def perk_define(row):
    if row['rank_hotel_hunter'] == row['min_c_h']:
        perk = 'Free Hotel Meal'
    elif row['rank_ave_bags'] == row['min_c_h']:
        perk = 'Free Checked Bag'
    elif row['rank_cancellation_rank'] == row['min_c_h']:
        perk = 'No Cancellation Fee'
    elif row['rank_bargain_hunter'] == row['min_c_h']:
        perk = 'Exclusive Discount'
    elif row['rank_combined_booking'] == row['min_c_h']:
        perk = 'Night Free Hotel with Flight'
    elif row['rank_session_activity'] == row['min_c_h']:
        perk = 'Active Searcher'
    else:
        'Not Defined'
    return perk



In [ ]:
# Step 4: Applying our function to determine customers' ranks:
indexes['most_attractive_perk'] = indexes.apply(perk_define, axis=1)
indexes.head()

,user_id,hotel_hunter_index,average_bags_scaled,cancellation_rate_scaled,bargain_hunter_index,combined_booking_scaled,session_intencity_index,rank_hotel_hunter,rank_ave_bags,rank_cancellation_rank,rank_bargain_hunter,rank_combined_booking,rank_session_activity,min_c_h,most_attractive_perk
0,23557,0.00268,NaN,0.0,NaN,0.0,0.333525,263.0,NaN,3069.0,NaN,5283.5,4866.5,263.0,Free Hotel Meal
1,94883,NaN,0.10,0.0,NaN,1.0,0.418203,NaN,2866.5,3069.0,NaN,1233.0,2687.0,1233.0,Night Free Hotel with Flight
2,101486,NaN,0.00,0.0,NaN,0.5,0.549539,NaN,4635.0,3069.0,NaN,4343.5,310.0,310.0,Active Searcher
3,101961,NaN,0.08,0.0,0.000058,1.0,0.176267,NaN,3380.0,3069.0,1679.0,1233.0,5988.0,1233.0,Night Free Hotel with Flight
4,106907,NaN,1.00,0.5,NaN,1.0,0.434310,NaN,4.0,52.0,NaN,1233.0,2231.0,4.0,Free Checked Bag


In [ ]:
# Let's see how many users in each segment
indexes['most_attractive_perk'].value_counts()

Active Searcher                 1477
Free Checked Bag                1096
Free Hotel Meal                  955
Exclusive Discount               905
Night Free Hotel with Flight     807
No Cancellation Fee              758
Name: most_attractive_perk, dtype: int64

In [ ]:
# Step 5: Downloading the file for further analysis in Tableau:
indexes.to_csv('Data_with_Perks.csv')

# Part 2. Processing users data

In [ ]:
# Loading the data from csv file with users data:
users_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/users_data.csv')
users_data.head()

,user_id,birthdate,gender,married,has_children,home_country,home_city,home_airport,home_airport_lat,home_airport_lon,sign_up_date
0,94883,1972-03-16,F,True,False,usa,kansas city,MCI,39.297,-94.714,2022-02-07
1,101486,1972-12-07,F,True,True,usa,tacoma,TCM,47.138,-122.476,2022-02-17
2,101961,1980-09-14,F,True,False,usa,boston,BOS,42.364,-71.005,2022-02-17
3,106907,1978-11-17,F,True,True,usa,miami,TNT,25.862,-80.897,2022-02-24
4,118043,1972-05-04,F,False,True,usa,los angeles,LAX,33.942,-118.408,2022-03-10


In [ ]:
users_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5998 entries, 0 to 5997
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           5998 non-null   int64  
 1   birthdate         5998 non-null   object 
 2   gender            5998 non-null   object 
 3   married           5998 non-null   bool   
 4   has_children      5998 non-null   bool   
 5   home_country      5998 non-null   object 
 6   home_city         5998 non-null   object 
 7   home_airport      5998 non-null   object 
 8   home_airport_lat  5998 non-null   float64
 9   home_airport_lon  5998 non-null   float64
 10  sign_up_date      5998 non-null   object 
dtypes: bool(2), float64(2), int64(1), object(6)
memory usage: 433.6+ KB


We have identified a few issues in our dataset that require attention:

1. Convert 'Birthdate' to date format.
2. Calculate the age based on 'Birthdate'.
3. Check the 'Gender' column for the number of unique values and consider converting this data to boolean.


In [ ]:
# Convert Birthdate to date format:
users_data['birthdate'] = pd.to_datetime(users_data['birthdate'], errors='coerce')

# Calculating age:
users_data['age'] = (pd.Timestamp.now() - users_data['birthdate'])//np.timedelta64(1,'Y')
users_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5998 entries, 0 to 5997
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   user_id           5998 non-null   int64         
 1   birthdate         5998 non-null   datetime64[ns]
 2   gender            5998 non-null   object        
 3   married           5998 non-null   bool          
 4   has_children      5998 non-null   bool          
 5   home_country      5998 non-null   object        
 6   home_city         5998 non-null   object        
 7   home_airport      5998 non-null   object        
 8   home_airport_lat  5998 non-null   float64       
 9   home_airport_lon  5998 non-null   float64       
 10  sign_up_date      5998 non-null   object        
 11  age               5998 non-null   int64         
dtypes: bool(2), datetime64[ns](1), float64(2), int64(2), object(5)
memory usage: 480.4+ KB


In [ ]:
# Cheking values in column Gender
users_data['gender'].value_counts()

F    5292
M     695
O      11
Name: gender, dtype: int64

In [ ]:
# Since amount of O-values is insignificant (only 0.18% of total) we will replace them by most frequent
# value ("F") and convert this column to boolean

replace_values = {'F' : True, 'O' : True, 'M' : False }
users_data['gender'] = users_data['gender'].map(replace_values)

In [ ]:
# We will drop unnecessary columns from the 'users_data' DataFrame to keep only the relevant information.
users_data.drop(['birthdate','home_country',
                 'home_city', 'home_airport',
                 'home_airport_lat', 'home_airport_lon',
                 'sign_up_date'], axis='columns', inplace=True)

In [ ]:
users_data.head()

,user_id,gender,married,has_children,age
0,94883,True,True,False,51
1,101486,True,True,True,50
2,101961,True,True,False,42
3,106907,True,True,True,44
4,118043,True,False,True,51


In [ ]:
# Downloading file for further analysis in Tableau:
users_data.to_csv('/content/drive/MyDrive/Colab Notebooks/users_data_processed.csv')